In [1]:
import re 
import datetime
import pandas as pd

import urllib.request as urllib_request
from urllib.error import URLError, HTTPError

from utils.helpers import *
from utils.functions import *
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [2]:
URL = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
soup = create_bsoup_object(URL)
tickers = get_tickers(soup)

In [3]:
cards = []
for ticker in tickers:
    url = 'https://fiis.com.br/' + ticker
    soup = create_bsoup_object(url)
    items = soup.find_all(class_ = 'value')
    infos = []
    card = {}
    for item in items:
        infos.append(item.get_text())
    card['ticker'] = ticker.upper()
    admin = soup.findAll("div", {"class": "text-wrapper"})
    card['administrador'] = re.sub("ADMINISTRADOR|[0-9].*[0-9]", "", admin[0].get_text())
    card['dividend_yield'] = infos[0]
    card['ultimo_rendimento'] = infos[1]
    card['patrimonio_liquido'] = infos[2]
    card['valor_patrimonial_por_cota'] = infos[3]
    card['telefone'] = infos[4]
    card['email'] = infos[5]
    card['site'] = infos[6]
    card['name'] = infos[7]
    card['tipo_FII'] = infos[8]
    card['tipo_ANBIMA'] = infos[9]
    card['registro_CVM'] = infos[10]
    card['numero_cotas'] = infos[11]
    card['numero_cotistas'] = infos[12]
    card['CNPJ'] = infos[13]
    card['cotacao'] = infos[14]
    card['min_52_weeks'] = infos[15]
    card['max_52_weeks'] = infos[16]
    card['return_12_months'] = infos[17]
        
    cards.append(card)

In [ ]:
## Create dataframe with the results
dataset = pd.DataFrame(cards)

In [ ]:
dataset[dataset.ticker == "BTAG11"]

,ticker,administrador,dividend_yield,ultimo_rendimento,patrimonio_liquido,valor_patrimonial_por_cota,telefone,email,site,name,tipo_FII,tipo_ANBIMA,registro_CVM,numero_cotas,numero_cotistas,CNPJ,cotacao,min_52_weeks,max_52_weeks,return_12_months,date
45,BTAG11,BTG Pactual,0.0077,0.75,"R$0,00",0.0,21 3262-9600,[email protected],www.btgpactual.com,FIAGRO BTAG,FIAGRO,Não Informado,nan,0.0,0.0,40.771.109/0001-47,98.0,91.6,103.65,0.0545,2022-05-21


In [ ]:
COLUMNS_LIST = ['ultimo_rendimento',
                'valor_patrimonial_por_cota',
                'numero_cotas',
                'numero_cotistas',
                'cotacao',
                'min_52_weeks',
                'max_52_weeks']
dataset = convert_string_to_float(dataset, COLUMNS_LIST)
dataset = convert_percentages_to_float(dataset, ['dividend_yield', 'return_12_months'])
dataset['registro_CVM'] = dataset['registro_CVM'].apply(lambda x: str(convert_to_datetime(x)))
#dataset["p_vp"] = (dataset['cotacao'] / dataset['valor_patrimonial_por_cota']).replace(np.inf, np.nan)
dataset['date'] = str(datetime.date.today())
dataset.head()

,ticker,administrador,dividend_yield,ultimo_rendimento,patrimonio_liquido,valor_patrimonial_por_cota,telefone,email,site,name,tipo_FII,tipo_ANBIMA,registro_CVM,numero_cotas,numero_cotistas,CNPJ,cotacao,min_52_weeks,max_52_weeks,return_12_months,date
0,ABCP11,Rio Bravo Investimentos,0.0089,0.65,"R$1,12 B",91.83,(11) 3509-6600,[email protected],www.riobravo.com.br,FII ABC IMOB,Tijolo: Shoppings,Renda Gestão Passiva,1996-05-15 00:00:00,12203832.0,19040.0,01.201.140/0001-90,72.80,60.01,78.27,0.0502,2022-05-21
1,AFHI11,BTG Pactual,0.0137,1.35,"R$172,46 M",96.47,21 3262-9600,[email protected],www.btgpactual.com,FII AFHI CRI,Papel: CRIs,Títulos e Valores Mobiliários Gestão Ativa,2020-10-20 00:00:00,1787671.0,7700.0,36.642.293/0001-58,98.25,88.00,103.45,0.0679,2022-05-21
2,AFOF11,BRL Trust DTVM,0.0128,1.14,"R$70,55 M",102.07,(11)31330350,[email protected],www.brltrust.com.br,FII ALIANZFF,Papel: Fundo de Fundos,Títulos e Valores Mobiliários Gestão Ativa,2020-01-22 00:00:00,691192.0,2584.0,34.847.063/0001-08,88.80,75.05,300.00,0.6036,2022-05-21
3,AIEC11,MAF DTVM,0.0091,0.72,"R$491,83 M",101.93,3223-7700,[email protected],www.modal.com.br,FII AUTONOMY,Tijolo: Escritórios,Renda Gestão Ativa,2020-08-24 00:00:00,4824987.0,6762.0,35.765.826/0001-26,78.99,67.50,88.00,0.1023,2022-05-21
4,ALMI11,BTG Pactual,0.0004,0.37,"R$250,49 M",2253.05,21 3262-9600,[email protected],www.btgpactual.com,FII TORRE AL,Tijolo: Escritórios,Renda Gestão Passiva,2006-01-03 00:00:00,111177.0,3114.0,07.122.725/0001-00,932.03,830.00,1300.00,0.1823,2022-05-21


In [ ]:
(dataset.cotacao / dataset.valor_patrimonial_por_cota).replace(np.inf, np.nan)

nan

In [ ]:
dataset[dataset.ticker == "BTAG11"]

,ticker,administrador,dividend_yield,ultimo_rendimento,patrimonio_liquido,valor_patrimonial_por_cota,telefone,email,site,name,tipo_FII,tipo_ANBIMA,registro_CVM,numero_cotas,numero_cotistas,CNPJ,cotacao,min_52_weeks,max_52_weeks,return_12_months,date
45,BTAG11,BTG Pactual,0.0077,0.75,"R$0,00",0.0,21 3262-9600,[email protected],www.btgpactual.com,FIAGRO BTAG,FIAGRO,Não Informado,nan,0.0,0.0,40.771.109/0001-47,98.0,91.6,103.65,0.0545,2022-05-21


## DynamoDB

In [5]:
import boto3
import json
from decimal import Decimal

In [123]:
ticker_data_dict = dataset.to_dict(orient='records')


In [6]:
dynamodb_resource = boto3.resource("dynamodb")


In [ ]:
batch_write(dynamodb_resource, "fiis", ticker_data_dict)

In [7]:
table = dynamodb_resource.Table("fiis")
response = table.scan()
df = pd.DataFrame(response['Items'])
df.head()

,patrimonio_liquido,administrador,ticker,return_12_months,telefone,site,CNPJ,email,registro_CVM,name,ultimo_rendimento,date,cotacao,numero_cotistas,max_52_weeks,valor_patrimonial_por_cota,numero_cotas,tipo_ANBIMA,dividend_yield,min_52_weeks,tipo_FII
0,"R$1,41 M",Oliveira Trust,LATR11B,0.8733,(21) 3514-0000,www.oliveiratrust.com.br,17.209.378/0001-00,[email protected],2012-11-19 00:00:00,FII LATERES,14.5,2022-05-22,21.16,164,167,11.33,124104,Desenvolvimento para Venda Gestão Ativa,0.6854,21.16,Tijolo: Desenvolvimento
1,"R$425,21 M",Hedge Investments DTVM LTDA,HREC11,0.0487,(11) 3124-4100,www.hedgeinvest.com.br,35.507.262/0001-21,[email protected],2019-11-21 00:00:00,FII HEDGEREC,1,2022-05-22,95.94,395,102,91.31,4656688,Títulos e Valores Mobiliários Gestão Ativa,0.0104,90.99,Papel: CRIs
2,"R$75,16 M",Banco Genial,PLOG11,0.2519,(21) 3923-3000,http://www.bancogenial.com/,36.501.198/0001-34,[email protected],2020-12-01 00:00:00,FII PLURAL L,0.75,2022-05-22,70.7,2356,95.43,100.21,750000,Renda Gestão Ativa,0.0106,68,Tijolo: Galpões
3,"R$86,25 M",BRL Trust DTVM,VCRI11,0.004,(11)31330350,www.brltrust.com.br,41.081.374/0001-66,[email protected],2021-03-23 00:00:00,FII VINCI CR,0.15,2022-05-22,9.99,1,10.69,10.34,8339175,Títulos e Valores Mobiliários Gestão Ativa,0.015,9.52,Papel: CRIs
4,"R$57,65 M",Oliveira Trust,ERPA11,0.0929,(21) 3514-0000,www.oliveiratrust.com.br,31.469.385/0001-64,[email protected],2019-02-25 00:00:00,FII EUROPA,0.31,2022-05-22,127,14,140,115.09,500920,Renda Gestão Passiva,0.0024,115,Tijolo: Escritórios


In [8]:
df.shape

(359, 21)